In [1]:
import os
import pandas as pd
import numpy as np
from scipy import sparse

import xgboost as xg
import lightgbm as lg
from sklearn.ensemble import RandomForestClassifier

from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.metrics import f1_score
import random

import warnings
warnings.filterwarnings('ignore')

C:\Users\DELL\Miniconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
os.environ['PYTHONHASHSEED']=str(2020)
np.random.seed(2020)
random.seed(2020)

In [3]:
def acc(ypred1,ypred2,ytest1,ytest2):
    f1 = f1_score(ypred1,ytest1,average='weighted')
    f2 = f1_score(ypred2,ytest2,average='weighted')
    s=(f1+f2)/2
    print(s)

In [4]:
train = pd.read_csv('train.csv',parse_dates=['issue_date','listing_date'])
test = pd.read_csv('test.csv',parse_dates=['issue_date','listing_date'])
train.shape , test.shape

((18834, 11), (8072, 9))

In [5]:
train.condition.value_counts(dropna=False)

1.0    6819
0.0    6281
2.0    4257
NaN    1477
Name: condition, dtype: int64

In [13]:
#train = train.sample(frac=1).reset_index(drop=True)

#combining train and test data
all_data = train.append(test)

#filling the missing the value of condition
all_data.fillna(3,inplace=True)

all_data[]

all_data['no_of_days'] =abs((all_data['issue_date'] - all_data['listing_date']).dt.days)
all_data['area'] = all_data['height(cm)'] * all_data['length(m)']*100

all_data['X1X2'] = (all_data['X1'].astype(str)+'_'+ all_data['X2'].astype(str))

all_data['count'] = all_data.groupby(["color_type"])["pet_id"].transform('count')

all_data.drop(['pet_id'],inplace=True,axis=1)
all_data.shape

(26906, 14)

In [172]:
all_data.head(3)

,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category,count,no_of_days,area,X1X2
0,2010-06-21,2017-12-14 12:52:00,0.0,Black,0.31,10.28,7,1,1.0,2.0,4620.0,2734,318.68,7_1
1,2014-12-20,2017-12-16 00:00:00,0.0,Fawn,0.58,45.30,13,9,1.0,2.0,159.0,1092,2627.40,13_9
2,2017-12-23,2019-01-20 00:00:00,2.0,Gray Tabby,0.39,29.15,13,9,1.0,1.0,65.0,393,1136.85,13_9


In [7]:
cat = list(all_data.select_dtypes('object'))
print('categorical values',cat)
for i in cat:
    l = LabelEncoder()
    all_data[i]= l.fit_transform(all_data[i])
    
all_data = pd.get_dummies(all_data,columns=['color_type',#'id_bins', 
                                   #'height_bin'
                                  ],drop_first=True)

#categorical = pd.get_dummies(categorical,drop_first=True)
#categorical =  sparse.csr_matrix(categorical)
all_data.shape

categorical values ['color_type', 'X1X2']


(26906, 68)

In [11]:
#train.color_type.value_counts()

train.color_type.value_counts()

Black                4620
White                2453
Brown                1791
Brown Tabby          1687
Tan                  1349
Blue                  852
Orange Tabby          791
Red                   526
Brown Brindle         496
Tricolor              469
Blue Tabby            386
Tortie                366
Calico                343
Gray                  307
Chocolate             259
Torbie                242
Cream Tabby           191
Sable                 167
Cream                 162
Fawn                  159
Yellow                143
Buff                  125
Lynx Point            117
Blue Merle            104
Seal Point             78
Black Brindle          66
Gray Tabby             65
Black Tabby            55
Flame Point            52
Brown Merle            39
Orange                 39
Black Smoke            32
Gold                   31
Tortie Point           26
Silver                 24
Red Tick               23
Blue Tick              21
Blue Point             20
Lilac Point 

In [176]:
train.head()`

SyntaxError: invalid syntax (<ipython-input-176-1eef4e98b7cb>, line 1)

In [177]:
y1 = train[['breed_category']]
y2 = train[['pet_category']]
drop1 = ['breed_category','pet_category',
        'issue_date', 'listing_date'
       ]
drop2 = ['pet_category',
        'issue_date', 'listing_date']
all1 = all_data.drop(drop1,axis=1)
all2 = all_data.drop(drop2,axis=1)
test_pet_id = test['pet_id']
cols1= list(all1.columns)
cols2 = list(all2.columns)

In [178]:
train.shape

(18834, 12)

In [179]:
train1 = all1.iloc[:train.shape[0],:]
test1 = all1.iloc[train.shape[0]:,:]
train2 = all2.iloc[:train.shape[0],:]
test2 = all2.iloc[train.shape[0]:,:]
train1.shape , test1.shape , train2.shape , test2.shape

((18834, 64), (8072, 64), (18834, 65), (8072, 65))

In [180]:

xtrain1 , xtest1 , ytrain1 , ytest1 = model_selection.train_test_split(train1,y1,
                                                                   random_state=2020,test_size=0.2)

xtrain2 , xtest2 , ytrain2 , ytest2 = model_selection.train_test_split(train2,y2,
                                                                   random_state=2020,test_size=0.2)

print('training data', xtrain1.shape,ytrain1.shape)
print('validation data ',xtest1.shape,ytest1.shape)

training data (15067, 64) (15067, 1)
validation data  (3767, 64) (3767, 1)


In [181]:
xgb = xg.XGBClassifier(random_state=2020)
xgb.fit(xtrain1,ytrain1)
ypred1 = xgb.predict(xtest1)
y1 = xgb.predict(test1)
xgb = xg.XGBClassifier(random_state=2020)
xgb.fit(xtrain2,ytrain2)
ypred2 = xgb.predict(xtest2)
test2['breed_category'] = y1
y2 = xgb.predict(test2)

acc(ypred1,ypred2,ytest1,ytest2)

0.9046754331004385


In [182]:
y2

array([2, 1, 2, ..., 2, 4, 1], dtype=int64)

In [183]:
df = pd.DataFrame({
    'pet_id':test_pet_id,
    'breed_category':y1,
    'pet_category':y2})
df.to_csv('submission.csv',index=False)

In [184]:
lgb = lg.LGBMClassifier(random_state=2020)
lgb.fit(xtrain1,ytrain1)
ypred1 = lgb.predict(xtest1)
y11 = lgb.predict(test1)
lgb = lg.LGBMClassifier(random_state=2020)
lgb.fit(xtrain2,ytrain2)
ypred2 = lgb.predict(xtest2)
test2['breed_category'] =y11
y22 = lgb.predict(test2)
acc(ypred1,ypred2,ytest1,ytest2)

0.9040418379244479


In [146]:
df = pd.DataFrame({
    'pet_id':test_pet_id,
    'breed_category':y11,
    'pet_category':y22})
df.to_csv('submission.csv',index=False)